Voglio customizzare il prompt ReAct in base al caso d'uso del chatbot, vedi [link](https://towardsdatascience.com/using-langchain-react-agents-for-answering-multi-hop-questions-in-rag-systems-893208c1847e)

In [1]:
import os
import telepot
from telepot.loop import MessageLoop
import datetime
import time
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import SystemMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
import re
from langchain.agents import create_tool_calling_agent
from dotenv import load_dotenv
load_dotenv(override=True)
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [2]:
#Tools
from tools.retrieval_eventi import get_relevant_document_tool 
from tools.tavily import web_search_tool 
from tools.utils import get_today_date_tool, get_summarized_text_tool
from tools.aws import ec2_shutdown_tools, ec2_turnon_tools
from prompts.react import prompt_react
from tools.google import get_calendar_events


c:\Users\ELAFACRB1\Codice\GitHub\langchain-rag-agent-chatbot\.venv\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 4800.07it/s]


In [3]:
tools = [get_relevant_document_tool, get_today_date_tool, web_search_tool, ec2_shutdown_tools, ec2_turnon_tools, get_calendar_events]

In [4]:
# new_prompt_react="""
# Use the context given to you to answer the question.
# When you come across time based questions like Has there been increase in sales from April last year and August last year,
# use today's date as {today_date} for fetching exact start and end dates. Then your first action should be to fetch the sales data with metadata time between April 1st to April 30th.
# That will be your answer 1. Your next action will be to fetch the sales data with metadata time between August 1st to
# August 31st, this will be your answer 2. Now analyse and understand each of answer 1 and answer 2, calculate the
# percentange change, compare them and answer the question.

# You have access to the following tools:

# {tools}
# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}
# Thought:{agent_scratchpad}
# """

In [5]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [6]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [7]:
system_template = 'Sei un aiutante servizievole.'
system_message_prompt_template = SystemMessagePromptTemplate.from_template(
    system_template)
prompt.messages[0]=system_message_prompt_template

In [8]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Sei un aiutante servizievole.')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [9]:
model = ChatGroq(model_name="llama3-70b-8192", groq_api_key=GROQ_API_KEY, temperature=0)
agent = create_tool_calling_agent(model, tools, prompt)

In [10]:
react_agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors=True
)

In [11]:
res = react_agent_executor.invoke({"input": "quali sno i prossimi 2 eventi nel mio calendario?"})



> Entering new AgentExecutor chain...

Invoking: `get_calendar_events` with `{'num_events': 2}`


[{'kind': 'calendar#event', 'etag': '"3430018974216000"', 'id': 'c8s32dhjc9hj4bb670pj4b9k6dj36bb1c4rm6b9k75h34p9l6cr68dpi6s', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=YzhzMzJkaGpjOWhqNGJiNjcwcGo0YjlrNmRqMzZiYjFjNHJtNmI5azc1aDM0cDlsNmNyNjhkcGk2cyBsYWZhY2Vyb3NhcmlvQG0', 'created': '2024-05-06T14:03:35.000Z', 'updated': '2024-05-06T15:31:27.108Z', 'summary': 'Salone auto Torino', 'location': 'Torino TO, Italia', 'creator': {'email': 'lafacerosario@gmail.com', 'self': True}, 'organizer': {'email': 'lafacerosario@gmail.com', 'self': True}, 'start': {'date': '2024-09-13'}, 'end': {'date': '2024-09-16'}, 'transparency': 'transparent', 'iCalUID': 'c8s32dhjc9hj4bb670pj4b9k6dj36bb1c4rm6b9k75h34p9l6cr68dpi6s@google.com', 'sequence': 0, 'attendees': [{'email': 'tropeatania@gmail.com', 'responseStatus': 'accepted'}, {'email': 'lafacerosario@gmail.com', 'organizer'

In [12]:
res['output']

'I see that the tool call yielded two events from your calendar. Here are the details of the next two events:\n\n1. **Salone auto Torino**: This event is scheduled to take place from September 13, 2024, to September 16, 2024, in Torino, Italy.\n2. **Update**: This event is a Microsoft Teams meeting scheduled for September 13, 2024, from 17:00 to 18:00 (CEST) with the meeting ID 333 056 454 011 and passcode w7kNpK.\n\nLet me know if you need any further assistance!'

In [13]:
res = react_agent_executor.invoke({"input": "cos'è un'euristica?"})
res['output']



> Entering new AgentExecutor chain...



Invoking: `Web eventi` with `euristica`


Collection web-places already exists!
20/10/2021      COSA CERCHICOSA CERCHI Seleziona una categoria AI News Bere&mangiare    Come?       All’aperto       Carne o pesce       Informale       Italiano       Kids Friendly       Occasione speciale       Pet friendly       Tiratardi       Uscite di gruppo&feste    Compilation    Cosa?       Carne       Cinese       Dolci&Pane       Gastronomie&Alimentari       Gelato       Giapponese       Internazionale       Pesce       Pizza       Street Food       Trattorie&Osterie       Veggie&Gluten Free       Vino, birra & drink    Delivery    Dove?       Arco&Sempione       Brera&Garibaldi       Cadorna&Sant’Ambrogio       Centrale&Repubblica       Cinque Giornate       Citt√† Studi       Duomo&San Babila       Isola       Lambrate       Loreto&Nolo       Navigli&Porta Genova       Porta Romana& Lodi       Porta Venezia       SantAgostino&Solari       Sarpi&Monumentale       Un po’ pi√π fuori       Wagner&

"L'euristica è la parte dell'epistemologia e del metodo scientifico che si occupa di favorire la ricerca di nuovi sviluppi teorici e scoperte empiriche. È l'arte di inventare, la ricerca di strategie, metodi e criteri che permettono di risolvere problemi attraverso la creatività, il pensiero divergente o laterale. Le euristiche sono atajos intuitivi che permettono di risolvere problemi e prendere decisioni rapidamente quando c'è informazione limitata."